#  <font color='red'>Table of Contents</font>


[13. Ensemble Model(CPU)](#section13)<br>


In [2]:
#ref - https://stackoverflow.com/questions/14463277/how-to-disable-python-warnings

# Basic Libs..
import warnings
warnings.filterwarnings("ignore")
import math
import time
import pandas as pd
import numpy as np
import scipy
import scipy.sparse
from tqdm import tqdm,tqdm_notebook
from prettytable import PrettyTable
import pickle
from IPython.display import Image
from contextlib import contextmanager
import os
print('CWD is ',os.getcwd())

# Vis Libs..
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud

# Text Libs..
import re
from gensim import corpora, models
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer,PorterStemmer
from nltk.stem.porter import *
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer

# ML Libs...
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder,FunctionTransformer
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
from sklearn.feature_selection import SelectKBest,f_regression
from sklearn.model_selection import GridSearchCV


# DL Libs..
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Input , Dropout, Flatten,concatenate,LSTM
from tensorflow.keras.layers import Embedding
import tensorflow.keras
from tensorflow.keras.models import Model,load_model
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import ModelCheckpoint,TensorBoard,ReduceLROnPlateau, EarlyStopping
from tensorflow.keras import backend as K
from tensorflow.keras import optimizers

# Colab Libs...
#from pydrive.auth import GoogleAuth
#from pydrive.drive import GoogleDrive
#from google.colab import auth
#from oauth2client.client import GoogleCredentials

CWD is  C:\Users\demitra\Downloads\mercari


#  <a id = 'section13'> <font color='red'>  13. Ensemble Model (CPU)  </font> </a>

### <font color='red'> 13.1 Load Data From GDrive </font>

In [3]:
file = open('df_train_model', 'rb')
df_train_model = pickle.load(file)
file.close()

file = open('df_test_model', 'rb')
df_test_model = pickle.load(file)
file.close()

print(df_train_model.shape,df_test_model.shape)

(1111901, 8) (370634, 8)


In [4]:
# ref - https://www.kaggle.com/valkling/mercari-rnn-2ridge-models-with-notes-0-42755

df_train_model = df_train_model[(df_train_model.price >= 3) & (df_train_model.price <= 2000)]
print(df_train_model.shape)

(1111222, 8)


### <font color='red'> 13.2 Data Preparation </font>

In [5]:
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

# https://gist.github.com/sebleier/554280
# we are removing the words from the stop words list: 'no', 'nor', 'not'
stopwords= ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"]

def text_preprocess(data):
    preprocessed = []
    # tqdm is for printing the status bar
    for sentance in tqdm_notebook(data):
        sent = decontracted(sentance)
        sent = sent.replace('\\r', ' ')
        sent = sent.replace('\\"', ' ')
        sent = sent.replace('\\n', ' ')
        sent = re.sub('[^A-Za-z0-9]+', ' ', sent)
        # https://gist.github.com/sebleier/554280
        sent = ' '.join(e for e in sent.split() if e not in stopwords)
        preprocessed.append(sent.lower().strip())
    return preprocessed

In [12]:
def text_encoder(train,test,vect_type,params):
    vectorizer = CountVectorizer(ngram_range = params[0],min_df = params[1],max_df = params[2],max_features = params[3],token_pattern = '\w+',
                                 dtype = np.float32) if vect_type == 'BOW' else TfidfVectorizer(ngram_range = params[0],min_df = params[1],
                                                                                                max_df = params[2],max_features = params[3],token_pattern = '\w+',
                                                                                                dtype = np.float32)
    train_transform = vectorizer.fit_transform(train)
    test_transform = vectorizer.transform(test)
    feat_names = vectorizer.get_feature_names()
    return train_transform,test_transform,feat_names

In [6]:
def dummy_encoder(train,test):
    train_transform = scipy.sparse.csr_matrix(pd.get_dummies(train[["item_condition_id", 
                                                                         "shipping"]], sparse = True).values)
    test_transform = scipy.sparse.csr_matrix(pd.get_dummies(test[["item_condition_id", 
                                                                         "shipping"]], sparse = True).values)
    return train_transform,test_transform

In [7]:
def brand_encoder(train,test):
    vect = LabelBinarizer(sparse_output=True)
    train_transform = vect.fit_transform(train["brand_name"])
    test_transform = vect.transform(test["brand_name"])
    return train_transform,test_transform

In [8]:
def category_encoder(train,test):
    unique_categories = pd.Series("/".join(train["category_name"].unique().astype("str")).split("/")).unique()
    count_category = CountVectorizer(vocabulary = unique_categories,lowercase = False,binary = True)
    train_transform = count_category.fit_transform(train["category_name"])
    test_transform = count_category.transform(test['category_name'])
    return train_transform,test_transform

In [9]:
def generate_cbs_stats(train,test):
    df_group = train.groupby('cat_brand_ship',as_index = False).agg({"shipping" : len,
                                                        "log_price" : [np.median, np.mean, np.std,np.min,np.max]})
    df_group.columns = ['cat_brand_ship','cbs_count','cbs_log_price_median','cbs_log_price_mean','cbs_log_price_std',
                    'cbs_log_price_min','cbs_log_price_max']

    df_group['cbs_log_price_std'] = df_group['cbs_log_price_std'].fillna(0)
    df_group['cbs_log_price_conf_variance'] = df_group['cbs_log_price_std'] / df_group['cbs_log_price_mean']
    df_group['cbs_log_count'] = np.log1p(df_group['cbs_count'])
    df_group['cbs_min_expected_log_price'] = (df_group['cbs_log_price_mean'] - (df_group['cbs_log_price_std']*2)).clip(lower=1.0)
    df_group['cbs_max_expected_log_price'] = (df_group['cbs_log_price_mean'] + (df_group['cbs_log_price_std']*2))
    
    df_group_stats = test.merge(df_group.reset_index(),
                                      how = 'left',
                                      on = 'cat_brand_ship')[['cbs_log_price_mean','cbs_min_expected_log_price','cbs_max_expected_log_price']].fillna(0).values
    
    scaler = StandardScaler(copy=True)
    cbs_feats_scaled = scaler.fit_transform(df_group_stats)
    return cbs_feats_scaled

In [13]:
train_bow_name,test_bow_name,bow_name_feats = text_encoder(text_preprocess(df_train_model['name'].replace([np.nan], '')),
                                                       text_preprocess(df_test_model['name'].replace([np.nan], '')),
                                                       'BOW',
                                                       ((1,3),25,1.0,50000))
train_bow_desc,test_bow_desc,bow_desc_feats = text_encoder(text_preprocess(df_train_model['item_description'].replace([np.nan,'No description yet'], '')),
                                                       text_preprocess(df_test_model['item_description'].replace([np.nan,'No description yet'], '')),
                                                       'BOW',
                                                       ((1,3),30,1.0,100000))
print(train_bow_name.shape,test_bow_name.shape,train_bow_desc.shape,test_bow_desc.shape)


(1111222, 32577) (370634, 32577) (1111222, 100000) (370634, 100000)


In [14]:
train_tfidf_name,test_tfidf_name,tfidf_name_feats = text_encoder(text_preprocess(df_train_model['name'].replace([np.nan], '')),
                                                       text_preprocess(df_test_model['name'].replace([np.nan], '')),
                                                       'TFIDF',
                                                       ((1,3),25,1.0,50000))
train_tfidf_desc,test_tfidf_desc,tfidf_desc_feats = text_encoder(text_preprocess(df_train_model['item_description'].replace([np.nan,'No description yet'], '')),
                                                       text_preprocess(df_test_model['item_description'].replace([np.nan,'No description yet'], '')),
                                                       'TFIDF',
                                                       ((1,3),30,1.0,100000))
print(train_tfidf_name.shape,test_tfidf_name.shape,train_tfidf_desc.shape,test_tfidf_desc.shape)


(1111222, 32577) (370634, 32577) (1111222, 100000) (370634, 100000)


In [15]:
train_brand,test_brand = brand_encoder(pd.DataFrame(df_train_model["brand_name"].fillna("missing").astype("category")),
                                       pd.DataFrame(df_test_model["brand_name"].fillna("missing").astype("category")))
print(train_brand.shape,test_brand.shape)

(1111222, 4445) (370634, 4445)


In [16]:
train_category,test_category = category_encoder(pd.DataFrame(df_train_model["category_name"].fillna("missing").astype("category")),
                                                pd.DataFrame(df_test_model["category_name"].fillna("missing").astype("category")))
print(train_category.shape,test_category.shape)

(1111222, 932) (370634, 932)


In [17]:
train_dummies,test_dummies = dummy_encoder(pd.DataFrame({"shipping" : df_train_model["shipping"].astype("category"),
                                                         "item_condition_id" : df_train_model["item_condition_id"].astype("category")}),
                                           pd.DataFrame({"shipping" : df_test_model["shipping"].astype("category"),
                                                         "item_condition_id" : df_test_model["item_condition_id"].astype("category")}))
print(train_dummies.shape,test_dummies.shape)

(1111222, 7) (370634, 7)


### <font color='red'> 13.3 Models </font>

In [18]:
def rmsle_score(y, y_pred):
    assert len(y) == len(y_pred)
    to_sum = [(math.log(y_pred[i] + 1) - math.log(y[i] + 1)) ** 2.0 for i,pred in enumerate(y_pred)]
    return (sum(to_sum) * (1.0/len(y))) ** 0.5

In [19]:
def rmsle_lgb(labels, preds):
    return 'rmsle', rmsle_score(preds, labels), False

In [20]:
def lgbm_model(X_train,y_train,X_test,y_test,params):
    model = lgb.LGBMRegressor(**params)
    model.fit(X_train, y_train,eval_set=[(X_test, y_test)],eval_metric=rmsle_lgb,early_stopping_rounds=None,verbose=True)
    return model

### <font color='red'> 13.4 Generate Ensembles </font>

**LGBM 1**

In [28]:
X_train = scipy.sparse.hstack((train_brand,train_category,train_dummies,train_bow_name,train_bow_desc)).tocsr().astype('float32')
X_test = scipy.sparse.hstack((test_brand,test_category,test_dummies,test_bow_name,test_bow_desc)).tocsr().astype('float32')

y_train = np.log1p(df_train_model['price'])
y_test = np.log1p(df_test_model['price'])
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)

lgbm = lgbm_model(X_train,y_train,X_test,y_test,{'n_estimators': 1500, 'learning_rate': 0.05, 'max_depth': -1,
               'num_leaves': 50, 'subsample': 0.8, 'colsample_bytree': 0.8,
               'min_child_samples': 50, 'n_jobs': -1})
preds_lgbm1 = np.expm1(lgbm.predict(X_test))
print('Test RMSLE = ',rmsle_score(np.expm1(y_test).values , preds_lgbm1),'\n')

file = open('preds_lgbm1', 'wb')
pickle.dump(preds_lgbm1, file)
file.close()

(1111222, 137961) (370634, 137961) (1111222,) (370634,)
[1]	valid_0's l2: 0.544642	valid_0's rmsle: 0.18506
[2]	valid_0's l2: 0.533397	valid_0's rmsle: 0.183137
[3]	valid_0's l2: 0.522009	valid_0's rmsle: 0.181183
[4]	valid_0's l2: 0.511716	valid_0's rmsle: 0.179394
[5]	valid_0's l2: 0.502241	valid_0's rmsle: 0.177722
[6]	valid_0's l2: 0.493451	valid_0's rmsle: 0.176173
[7]	valid_0's l2: 0.485398	valid_0's rmsle: 0.174739
[8]	valid_0's l2: 0.478085	valid_0's rmsle: 0.173448
[9]	valid_0's l2: 0.471286	valid_0's rmsle: 0.172234
[10]	valid_0's l2: 0.465157	valid_0's rmsle: 0.171126
[11]	valid_0's l2: 0.459447	valid_0's rmsle: 0.170071
[12]	valid_0's l2: 0.454236	valid_0's rmsle: 0.169085
[13]	valid_0's l2: 0.449494	valid_0's rmsle: 0.168216
[14]	valid_0's l2: 0.444976	valid_0's rmsle: 0.167389
[15]	valid_0's l2: 0.440481	valid_0's rmsle: 0.166546
[16]	valid_0's l2: 0.436646	valid_0's rmsle: 0.165824
[17]	valid_0's l2: 0.432512	valid_0's rmsle: 0.165058
[18]	valid_0's l2: 0.42857	valid_0's

[300]	valid_0's l2: 0.264427	valid_0's rmsle: 0.131015
[301]	valid_0's l2: 0.264282	valid_0's rmsle: 0.130982
[302]	valid_0's l2: 0.264145	valid_0's rmsle: 0.130954
[303]	valid_0's l2: 0.264047	valid_0's rmsle: 0.130931
[304]	valid_0's l2: 0.263915	valid_0's rmsle: 0.130903
[305]	valid_0's l2: 0.263796	valid_0's rmsle: 0.130876
[306]	valid_0's l2: 0.263653	valid_0's rmsle: 0.130843
[307]	valid_0's l2: 0.263556	valid_0's rmsle: 0.130821
[308]	valid_0's l2: 0.263422	valid_0's rmsle: 0.130791
[309]	valid_0's l2: 0.263313	valid_0's rmsle: 0.130767
[310]	valid_0's l2: 0.263206	valid_0's rmsle: 0.130743
[311]	valid_0's l2: 0.26309	valid_0's rmsle: 0.130717
[312]	valid_0's l2: 0.262976	valid_0's rmsle: 0.130689
[313]	valid_0's l2: 0.262879	valid_0's rmsle: 0.130667
[314]	valid_0's l2: 0.262742	valid_0's rmsle: 0.130637
[315]	valid_0's l2: 0.262616	valid_0's rmsle: 0.13061
[316]	valid_0's l2: 0.26248	valid_0's rmsle: 0.130579
[317]	valid_0's l2: 0.262379	valid_0's rmsle: 0.130555
[318]	valid_0

[598]	valid_0's l2: 0.242227	valid_0's rmsle: 0.12579
[599]	valid_0's l2: 0.242179	valid_0's rmsle: 0.125778
[600]	valid_0's l2: 0.242137	valid_0's rmsle: 0.125769
[601]	valid_0's l2: 0.242093	valid_0's rmsle: 0.125758
[602]	valid_0's l2: 0.242029	valid_0's rmsle: 0.125743
[603]	valid_0's l2: 0.241987	valid_0's rmsle: 0.125733
[604]	valid_0's l2: 0.241937	valid_0's rmsle: 0.125721
[605]	valid_0's l2: 0.241895	valid_0's rmsle: 0.12571
[606]	valid_0's l2: 0.241848	valid_0's rmsle: 0.1257
[607]	valid_0's l2: 0.241803	valid_0's rmsle: 0.125689
[608]	valid_0's l2: 0.241761	valid_0's rmsle: 0.125679
[609]	valid_0's l2: 0.24172	valid_0's rmsle: 0.125669
[610]	valid_0's l2: 0.241651	valid_0's rmsle: 0.125653
[611]	valid_0's l2: 0.241606	valid_0's rmsle: 0.125642
[612]	valid_0's l2: 0.241562	valid_0's rmsle: 0.125632
[613]	valid_0's l2: 0.241517	valid_0's rmsle: 0.125621
[614]	valid_0's l2: 0.241464	valid_0's rmsle: 0.125606
[615]	valid_0's l2: 0.241428	valid_0's rmsle: 0.125597
[616]	valid_0's

[896]	valid_0's l2: 0.231388	valid_0's rmsle: 0.123142
[897]	valid_0's l2: 0.231362	valid_0's rmsle: 0.123136
[898]	valid_0's l2: 0.231336	valid_0's rmsle: 0.123129
[899]	valid_0's l2: 0.231313	valid_0's rmsle: 0.123123
[900]	valid_0's l2: 0.231287	valid_0's rmsle: 0.123116
[901]	valid_0's l2: 0.231262	valid_0's rmsle: 0.12311
[902]	valid_0's l2: 0.231235	valid_0's rmsle: 0.123103
[903]	valid_0's l2: 0.231211	valid_0's rmsle: 0.123098
[904]	valid_0's l2: 0.231186	valid_0's rmsle: 0.123091
[905]	valid_0's l2: 0.231164	valid_0's rmsle: 0.123085
[906]	valid_0's l2: 0.23114	valid_0's rmsle: 0.123079
[907]	valid_0's l2: 0.231113	valid_0's rmsle: 0.123071
[908]	valid_0's l2: 0.231087	valid_0's rmsle: 0.123064
[909]	valid_0's l2: 0.23106	valid_0's rmsle: 0.123058
[910]	valid_0's l2: 0.231031	valid_0's rmsle: 0.123052
[911]	valid_0's l2: 0.230998	valid_0's rmsle: 0.123045
[912]	valid_0's l2: 0.230974	valid_0's rmsle: 0.123038
[913]	valid_0's l2: 0.230924	valid_0's rmsle: 0.123026
[914]	valid_0

[1190]	valid_0's l2: 0.22473	valid_0's rmsle: 0.12147
[1191]	valid_0's l2: 0.224712	valid_0's rmsle: 0.121466
[1192]	valid_0's l2: 0.224699	valid_0's rmsle: 0.121462
[1193]	valid_0's l2: 0.224678	valid_0's rmsle: 0.121457
[1194]	valid_0's l2: 0.224654	valid_0's rmsle: 0.121451
[1195]	valid_0's l2: 0.224638	valid_0's rmsle: 0.121447
[1196]	valid_0's l2: 0.224623	valid_0's rmsle: 0.121443
[1197]	valid_0's l2: 0.22461	valid_0's rmsle: 0.12144
[1198]	valid_0's l2: 0.224595	valid_0's rmsle: 0.121436
[1199]	valid_0's l2: 0.224579	valid_0's rmsle: 0.121432
[1200]	valid_0's l2: 0.224562	valid_0's rmsle: 0.121428
[1201]	valid_0's l2: 0.224547	valid_0's rmsle: 0.121424
[1202]	valid_0's l2: 0.224525	valid_0's rmsle: 0.121419
[1203]	valid_0's l2: 0.224512	valid_0's rmsle: 0.121416
[1204]	valid_0's l2: 0.224496	valid_0's rmsle: 0.121411
[1205]	valid_0's l2: 0.22448	valid_0's rmsle: 0.121408
[1206]	valid_0's l2: 0.224456	valid_0's rmsle: 0.121402
[1207]	valid_0's l2: 0.22444	valid_0's rmsle: 0.12139

[1483]	valid_0's l2: 0.219966	valid_0's rmsle: 0.120263
[1484]	valid_0's l2: 0.219955	valid_0's rmsle: 0.12026
[1485]	valid_0's l2: 0.219928	valid_0's rmsle: 0.120253
[1486]	valid_0's l2: 0.219912	valid_0's rmsle: 0.120249
[1487]	valid_0's l2: 0.219895	valid_0's rmsle: 0.120245
[1488]	valid_0's l2: 0.219882	valid_0's rmsle: 0.120242
[1489]	valid_0's l2: 0.219869	valid_0's rmsle: 0.120238
[1490]	valid_0's l2: 0.219854	valid_0's rmsle: 0.120234
[1491]	valid_0's l2: 0.21984	valid_0's rmsle: 0.120231
[1492]	valid_0's l2: 0.219826	valid_0's rmsle: 0.120227
[1493]	valid_0's l2: 0.219817	valid_0's rmsle: 0.120224
[1494]	valid_0's l2: 0.219805	valid_0's rmsle: 0.120221
[1495]	valid_0's l2: 0.219792	valid_0's rmsle: 0.120218
[1496]	valid_0's l2: 0.21978	valid_0's rmsle: 0.120215
[1497]	valid_0's l2: 0.219764	valid_0's rmsle: 0.12021
[1498]	valid_0's l2: 0.219755	valid_0's rmsle: 0.120208
[1499]	valid_0's l2: 0.219742	valid_0's rmsle: 0.120204
[1500]	valid_0's l2: 0.219722	valid_0's rmsle: 0.120

**LGBM 2**

In [30]:
X_train = scipy.sparse.hstack((train_brand,train_category,train_dummies,train_tfidf_name,train_tfidf_desc)).tocsr().astype('float32')
X_test = scipy.sparse.hstack((test_brand,test_category,test_dummies,test_tfidf_name,test_tfidf_desc)).tocsr().astype('float32')

y_train = np.log1p(df_train_model['price'])
y_test = np.log1p(df_test_model['price'])
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)

lgbm = lgbm_model(X_train,y_train,X_test,y_test,{'n_estimators': 1500, 'learning_rate': 0.05, 'max_depth': -1,
               'num_leaves': 50, 'subsample': 0.8, 'colsample_bytree': 0.8,
               'min_child_samples': 50, 'n_jobs': -1})
preds_lgbm2 = np.expm1(lgbm.predict(X_test))
print('Test RMSLE = ',rmsle_score(np.expm1(y_test).values , preds_lgbm2),'\n')

file = open('preds_lgbm2', 'wb')
pickle.dump(preds_lgbm2, file)
file.close()

(1111222, 137961) (370634, 137961) (1111222,) (370634,)
[1]	valid_0's l2: 0.54461	valid_0's rmsle: 0.185054
[2]	valid_0's l2: 0.53336	valid_0's rmsle: 0.18313
[3]	valid_0's l2: 0.52195	valid_0's rmsle: 0.181171
[4]	valid_0's l2: 0.511698	valid_0's rmsle: 0.179389
[5]	valid_0's l2: 0.502227	valid_0's rmsle: 0.177723
[6]	valid_0's l2: 0.493459	valid_0's rmsle: 0.176174
[7]	valid_0's l2: 0.48538	valid_0's rmsle: 0.17473
[8]	valid_0's l2: 0.478015	valid_0's rmsle: 0.173427
[9]	valid_0's l2: 0.471083	valid_0's rmsle: 0.172188
[10]	valid_0's l2: 0.464955	valid_0's rmsle: 0.171077
[11]	valid_0's l2: 0.459349	valid_0's rmsle: 0.170045
[12]	valid_0's l2: 0.453968	valid_0's rmsle: 0.169027
[13]	valid_0's l2: 0.449207	valid_0's rmsle: 0.168157
[14]	valid_0's l2: 0.44472	valid_0's rmsle: 0.167333
[15]	valid_0's l2: 0.4399	valid_0's rmsle: 0.166447
[16]	valid_0's l2: 0.435994	valid_0's rmsle: 0.165716
[17]	valid_0's l2: 0.431974	valid_0's rmsle: 0.164952
[18]	valid_0's l2: 0.428333	valid_0's rmsle:

[300]	valid_0's l2: 0.263745	valid_0's rmsle: 0.130828
[301]	valid_0's l2: 0.263633	valid_0's rmsle: 0.130802
[302]	valid_0's l2: 0.263518	valid_0's rmsle: 0.130774
[303]	valid_0's l2: 0.263414	valid_0's rmsle: 0.130749
[304]	valid_0's l2: 0.263308	valid_0's rmsle: 0.130724
[305]	valid_0's l2: 0.263171	valid_0's rmsle: 0.130691
[306]	valid_0's l2: 0.263052	valid_0's rmsle: 0.130664
[307]	valid_0's l2: 0.262917	valid_0's rmsle: 0.130632
[308]	valid_0's l2: 0.262786	valid_0's rmsle: 0.1306
[309]	valid_0's l2: 0.262652	valid_0's rmsle: 0.13057
[310]	valid_0's l2: 0.262554	valid_0's rmsle: 0.130548
[311]	valid_0's l2: 0.262436	valid_0's rmsle: 0.130518
[312]	valid_0's l2: 0.262308	valid_0's rmsle: 0.130491
[313]	valid_0's l2: 0.26221	valid_0's rmsle: 0.130468
[314]	valid_0's l2: 0.26211	valid_0's rmsle: 0.130445
[315]	valid_0's l2: 0.261989	valid_0's rmsle: 0.130418
[316]	valid_0's l2: 0.261867	valid_0's rmsle: 0.130388
[317]	valid_0's l2: 0.261763	valid_0's rmsle: 0.130364
[318]	valid_0's

[598]	valid_0's l2: 0.241748	valid_0's rmsle: 0.125633
[599]	valid_0's l2: 0.241703	valid_0's rmsle: 0.12562
[600]	valid_0's l2: 0.241655	valid_0's rmsle: 0.125608
[601]	valid_0's l2: 0.241606	valid_0's rmsle: 0.125599
[602]	valid_0's l2: 0.241558	valid_0's rmsle: 0.125588
[603]	valid_0's l2: 0.241517	valid_0's rmsle: 0.125578
[604]	valid_0's l2: 0.241475	valid_0's rmsle: 0.125567
[605]	valid_0's l2: 0.241411	valid_0's rmsle: 0.125553
[606]	valid_0's l2: 0.24137	valid_0's rmsle: 0.125543
[607]	valid_0's l2: 0.241301	valid_0's rmsle: 0.125525
[608]	valid_0's l2: 0.241263	valid_0's rmsle: 0.125516
[609]	valid_0's l2: 0.241196	valid_0's rmsle: 0.125501
[610]	valid_0's l2: 0.241158	valid_0's rmsle: 0.125491
[611]	valid_0's l2: 0.241118	valid_0's rmsle: 0.125482
[612]	valid_0's l2: 0.24107	valid_0's rmsle: 0.12547
[613]	valid_0's l2: 0.241031	valid_0's rmsle: 0.125459
[614]	valid_0's l2: 0.240991	valid_0's rmsle: 0.12545
[615]	valid_0's l2: 0.240946	valid_0's rmsle: 0.12544
[616]	valid_0's 

[896]	valid_0's l2: 0.231018	valid_0's rmsle: 0.123003
[897]	valid_0's l2: 0.230996	valid_0's rmsle: 0.122997
[898]	valid_0's l2: 0.230966	valid_0's rmsle: 0.122989
[899]	valid_0's l2: 0.230945	valid_0's rmsle: 0.122983
[900]	valid_0's l2: 0.230916	valid_0's rmsle: 0.122976
[901]	valid_0's l2: 0.230887	valid_0's rmsle: 0.122969
[902]	valid_0's l2: 0.230857	valid_0's rmsle: 0.122962
[903]	valid_0's l2: 0.230824	valid_0's rmsle: 0.122954
[904]	valid_0's l2: 0.2308	valid_0's rmsle: 0.122948
[905]	valid_0's l2: 0.23078	valid_0's rmsle: 0.122942
[906]	valid_0's l2: 0.230757	valid_0's rmsle: 0.122936
[907]	valid_0's l2: 0.230728	valid_0's rmsle: 0.122929
[908]	valid_0's l2: 0.230703	valid_0's rmsle: 0.122922
[909]	valid_0's l2: 0.23068	valid_0's rmsle: 0.122916
[910]	valid_0's l2: 0.230656	valid_0's rmsle: 0.12291
[911]	valid_0's l2: 0.230634	valid_0's rmsle: 0.122904
[912]	valid_0's l2: 0.230602	valid_0's rmsle: 0.122897
[913]	valid_0's l2: 0.230578	valid_0's rmsle: 0.122891
[914]	valid_0's

[1190]	valid_0's l2: 0.224418	valid_0's rmsle: 0.121347
[1191]	valid_0's l2: 0.224398	valid_0's rmsle: 0.121342
[1192]	valid_0's l2: 0.224374	valid_0's rmsle: 0.121336
[1193]	valid_0's l2: 0.22436	valid_0's rmsle: 0.121333
[1194]	valid_0's l2: 0.224345	valid_0's rmsle: 0.121329
[1195]	valid_0's l2: 0.224318	valid_0's rmsle: 0.121322
[1196]	valid_0's l2: 0.224301	valid_0's rmsle: 0.121318
[1197]	valid_0's l2: 0.224286	valid_0's rmsle: 0.121314
[1198]	valid_0's l2: 0.224269	valid_0's rmsle: 0.12131
[1199]	valid_0's l2: 0.224253	valid_0's rmsle: 0.121306
[1200]	valid_0's l2: 0.224236	valid_0's rmsle: 0.121302
[1201]	valid_0's l2: 0.224219	valid_0's rmsle: 0.121297
[1202]	valid_0's l2: 0.224204	valid_0's rmsle: 0.121293
[1203]	valid_0's l2: 0.224183	valid_0's rmsle: 0.121287
[1204]	valid_0's l2: 0.224145	valid_0's rmsle: 0.121277
[1205]	valid_0's l2: 0.224129	valid_0's rmsle: 0.121274
[1206]	valid_0's l2: 0.224113	valid_0's rmsle: 0.12127
[1207]	valid_0's l2: 0.224098	valid_0's rmsle: 0.12

[1482]	valid_0's l2: 0.219648	valid_0's rmsle: 0.120131
[1483]	valid_0's l2: 0.219632	valid_0's rmsle: 0.120127
[1484]	valid_0's l2: 0.219621	valid_0's rmsle: 0.120124
[1485]	valid_0's l2: 0.219606	valid_0's rmsle: 0.12012
[1486]	valid_0's l2: 0.219595	valid_0's rmsle: 0.120117
[1487]	valid_0's l2: 0.219584	valid_0's rmsle: 0.120114
[1488]	valid_0's l2: 0.21957	valid_0's rmsle: 0.12011
[1489]	valid_0's l2: 0.219557	valid_0's rmsle: 0.120107
[1490]	valid_0's l2: 0.219546	valid_0's rmsle: 0.120104
[1491]	valid_0's l2: 0.219534	valid_0's rmsle: 0.120101
[1492]	valid_0's l2: 0.219521	valid_0's rmsle: 0.120098
[1493]	valid_0's l2: 0.21951	valid_0's rmsle: 0.120095
[1494]	valid_0's l2: 0.219481	valid_0's rmsle: 0.120088
[1495]	valid_0's l2: 0.219464	valid_0's rmsle: 0.120084
[1496]	valid_0's l2: 0.219457	valid_0's rmsle: 0.120082
[1497]	valid_0's l2: 0.219445	valid_0's rmsle: 0.120078
[1498]	valid_0's l2: 0.219434	valid_0's rmsle: 0.120075
[1499]	valid_0's l2: 0.219418	valid_0's rmsle: 0.120